In [1]:
from fastai.text.all import *

In [2]:
lang = 'de'
path = Path('data/dewiki_lstm_15k')
model_path = path/'model'
spm_path = Path('data/spm_de_ft')
lm_fns = [model_path/f'{lang}_wikitext', model_path/f'{lang}_wikitext_vocab']

In [3]:
lm_fns[0] = lm_fns[0].absolute()
lm_fns[1] = lm_fns[1].absolute()

In [4]:
lm_fns

[Path('/data/projects/git/fastai_ulmfit_german/data/dewiki_lstm_15k/model/de_wikitext'),
 Path('/data/projects/git/fastai_ulmfit_german/data/dewiki_lstm_15k/model/de_wikitext_vocab')]

In [5]:
bs = 64

## Prepare corpus for fine tuning

In [6]:
names = ['text','label','label1,']

In [7]:
df_train = pd.read_csv('data/germeval2018/germeval2018.training.txt',
                sep ='\t', names=names)

In [8]:
df_valid = pd.read_csv('data/germeval2018/germeval2018.test.txt',
                sep ='\t', names=names)

In [9]:
df_train2 = pd.read_csv('data/germeval2019/germeval2019.training_subtask1_2_korrigiert.txt',
                sep = '\t', names=names)

In [10]:
df_train3 = pd.read_csv('data/germeval2019/germeval2019.training_subtask3.txt',
                sep = '\t', names=[*names,'label3'])
df_train3.drop('label3', axis=1)

,text,label,"label1,"
0,@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.,OFFENSE,INSULT
1,@milenahanm 33 bis 45 habe ich noch gar nicht gelebt und es geht mir am Arsch vorbei was in dieser Zeit geschehen ist. Ich lebe im heute und jetzt und nicht in der Vergangenheit.,OFFENSE,PROFANITY
2,@tagesschau Euere AfD Hetze wirkt. Da könnt ihr stolz sein bei #ARD-Fernsehen,OFFENSE,ABUSE
3,"Deutsche Medien, Halbwahrheiten und einseitige Betrachtung, wie bei allen vom Staat finanzierten ""billigen"" Propagandainstitutionen 😜",OFFENSE,ABUSE
4,@Ralf_Stegner Oman Ralle..dich mag ja immer noch keiner. Du willst das die Hetze gegen dich aufhört? |LBR| Geh in Rente und verzichte auf die 1/2deiner Pension,OFFENSE,INSULT
...,...,...,...
1916,@Alltags_Kotze Dein Feminismus und Genderquatsch steht Dir im Weg,OFFENSE,ABUSE
1917,@UdoUlfkotte Hauptsache den Asylanten gehts gesundheitlich gut. Deutsche Patienten(Rentner) können sehen wo sie bleiben.,OFFENSE,ABUSE
1918,"@SteinbachErika Ich finde AFD Wähler besser als fettige Hasenscharten, die auf Kosten aller permanent am schmarotzen sind.",OFFENSE,INSULT
1919,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ist eine Religion 😂😂😂",OFFENSE,ABUSE


In [11]:
df = pd.concat([df_train, df_valid,df_train2,df_train3], sort=False)

## Fine tune model

In [12]:
tok = SentencePieceTokenizer(lang=lang, max_vocab_sz=15000, cache_dir=spm_path)

In [13]:
dblocks = DataBlock(blocks=(TextBlock.from_df('text', tok=tok, is_lm=True)),
                    get_x=ColReader('text'), 
                    splitter=RandomSplitter(valid_pct=0.1, seed=42))
dls = dblocks.dataloaders(df)

/home/florian/miniconda3/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [33]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.7, pretrained=True, pretrained_fnames=lm_fns, 
                               metrics=[accuracy, Perplexity()]).to_native_fp16()
learn.path = model_path

In [34]:
#learn.lr_find()

In [35]:
lr = 1e-2
#lr *= bs/48  # Scale learning rate by batch size

In [36]:
learn.fit_one_cycle(1, lr, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.272661,4.807849,0.253456,122.467957,00:12


In [37]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(lr/100,lr), moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.727436,4.520339,0.280392,91.866692,00:13
1,4.386356,4.185076,0.307245,65.698479,00:13
2,4.065926,3.972458,0.324279,53.114910,00:13
3,3.755505,3.849533,0.336428,46.971138,00:13
4,3.522296,3.776676,0.346177,43.670658,00:13
5,3.339683,3.736617,0.352000,41.955795,00:13
6,3.163114,3.713682,0.358854,41.004505,00:13
7,3.042154,3.710827,0.360801,40.887592,00:13
8,2.962108,3.709983,0.361749,40.853127,00:13
9,2.915797,3.710767,0.362093,40.885166,00:13


## Saving fine tuned model, encoder and vocab

In [38]:
lm_ft_fns = [model_path/f'{lang}_ft', model_path/f'{lang}_ft_vocab.pkl']

In [39]:
learn.to_fp32()

In [40]:
learn.save(lm_ft_fns[0].absolute(), with_opt=False)

Path('/data/projects/git/fastai_ulmfit_german/data/dewiki_lstm_15k/model/de_ft.pth')

In [52]:
learn.save_encoder(f'{lm_ft_fns[0]}_encoder')

In [53]:
with open(lm_ft_fns[1], 'wb') as f:
      pickle.dump(learn.dls.vocab, f)